#  adversarial training using custom loss


In [681]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [682]:
import os
from os.path import join

import time 
import numpy as np
import pandas as pd
import cv2
import pickle
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
import gc


from tensorflow import convert_to_tensor
import tensorflow.keras.backend as K
from tensorflow.keras.models import  load_model,Model
from tensorflow.keras import applications
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout, Activation, Dense, GlobalMaxPooling2D,Conv2D,Flatten,MaxPooling2D,InputLayer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import Loss,CategoricalCrossentropy

import copy

from efficientnet.tfkeras import EfficientNetB7
from tensorflow.keras import datasets
from foolbox.attacks import LinfFastGradientAttack,LinfDeepFoolAttack
from foolbox.distances import LpDistance
from foolbox.models import TensorFlowModel
from foolbox import criteria
from sklearn.metrics import classification_report


from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import BoundaryAttack
from art.utils import load_dataset #to play with cifar images


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4*1024)]
)

In [683]:
tf.config.run_functions_eagerly(False)

In [700]:
tf.random.set_seed(0)

#### 1. Load Data

In [684]:
data=datasets.mnist.load_data(path='mnist.npz')
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# X_train,y_train=data[0][0],data[0][1]
# X_test,y_test=data[1][0],data[1][1]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

""" Train """
X_train_img = np.full((60000, 32, 32, 3), 0)
for i, s in enumerate(X_train):
    X_train_img[i] = cv2.cvtColor(np.pad(s,2), cv2.COLOR_GRAY2RGB)
    
""" Test """
X_test_img = np.full((10000, 32, 32, 3), 0)
for i, s in enumerate(X_test):
    X_test_img[i] = cv2.cvtColor(np.pad(s,2), cv2.COLOR_GRAY2RGB)
    
X_train=X_train_img
X_test= X_test_img

LABEL_NAMES = ['t_shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boots']

In [685]:
data_set=tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(buffer_size=100000).batch(32)

#### 2. Load models

In [686]:
class Model(tf.keras.Model):
    def __init__(self,c,learning_rate,epsilon):
        super(Model,self).__init__()
        
        self.c=c
        self.learning_rate=learning_rate
        self.epsilon=epsilon
        
        self.model = tf.keras.models.Sequential()
        self.model.add(Conv2D(64, (3, 3), activation='relu',input_shape=(32,32,3)))
        self.model.add(MaxPooling2D(2, 2))
        self.model.add(Conv2D(64, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(2, 2))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(10, activation='softmax'))

        self.loss = CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.SGD(self.learning_rate)
#         self.optimizer=tf.keras.optimizers.Adam(self.learning_rate)

        
        
   
    @tf.function
    def train_step(self,x,y):

        with tf.GradientTape() as tape_w:  
            
            with tf.GradientTape() as tape_x:
                tape_x.watch(x)
                true_loss = self.loss(y, self.model(x))        
            gradient = tape_x.gradient(true_loss, x)       
            signed_grad = tf.sign(gradient)      
            adv_loss= self.loss(y, self.model(x-self.epsilon*signed_grad))    
            
      # Combines both losses            
            total_loss= (1-self.c)*true_loss + self.c*adv_loss
            
    # Regular backward pass.
            gradients = tape_w.gradient(total_loss, self.model.trainable_variables)        
            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
            
       
    def train(self,train_dataset, x_test,y_test,epochs):

        pred=list(map(np.argmax,self.model.predict(x_test)))
        true_values=list(map(np.argmax,y_test))
        acc=np.sum([1 for i in range(len(pred)) if pred[i]==true_values[i]])/len(pred)         
        print('Accuracy before training is {} '.format(acc))
        print('-----------')

        for epoch in range(epochs):
            if epoch!=0 and epoch%3==0:
                self.epsilon*=10
            start = time.time()
            for (x,y) in tqdm(train_dataset,position=0):
                x=tf.cast(x,dtype='float32')           
                self.train_step(x,y)           

            pred=list(map(np.argmax,self.model.predict(x_test)))
            true_values=list(map(np.argmax,y_test))
            acc=np.sum([1 for i in range(len(pred)) if pred[i]==true_values[i]])/len(pred)         

            print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
            print('-----------')
            print('Accuracy for epoch {} is {} '.format(epoch + 1, acc))        
            
       
    def call(self, inputs):
        x = self.model(inputs)
        return x

    


        

In [687]:
model_with_def=Model(c=0.5,learning_rate=0.001,epsilon=0.001)
model_without_def=Model(c=0,learning_rate=0.001,epsilon=0.001)
model_slight_defense=Model(c=0.25,learning_rate=0.001,epsilon=0.001)
model_strong_defense=Model(c=0.75,learning_rate=0.001,epsilon=0.001)

In [702]:
model_average_defense=Model(c=0.6,learning_rate=0.001,epsilon=0.001)

In [688]:
model_with_def.train(data_set, X_test[:1000],y_test[:1000], 10)

  0%|          | 0/1875 [00:00<?, ?it/s]

Accuracy before training is 0.091 
-----------


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 1 is 11.521335124969482 sec
-----------
Accuracy for epoch 1 is 0.86 


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Time for epoch 8 is 11.051707744598389 sec
-----------
Accuracy for epoch 8 is 0.904 


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 9 is 10.72933840751648 sec
-----------
Accuracy for epoch 9 is 0.892 


100%|██████████| 1875/1875 [00:10<00:00, 172.97it/s]


Time for epoch 10 is 10.939248323440552 sec
-----------
Accuracy for epoch 10 is 0.898 


In [689]:
model_without_def.train(data_set, X_test[:1000],y_test[:1000], 10)

  0%|          | 0/1875 [00:00<?, ?it/s]

Accuracy before training is 0.119 
-----------


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 1 is 11.084503889083862 sec
-----------
Accuracy for epoch 1 is 0.859 


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Time for epoch 8 is 10.681093215942383 sec
-----------
Accuracy for epoch 8 is 0.897 


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 9 is 10.686333179473877 sec
-----------
Accuracy for epoch 9 is 0.9 


100%|██████████| 1875/1875 [00:10<00:00, 176.53it/s]


Time for epoch 10 is 10.719868898391724 sec
-----------
Accuracy for epoch 10 is 0.897 


In [690]:
model_slight_defense.train(data_set, X_test[:1000],y_test[:1000], 10)

  0%|          | 0/1875 [00:00<?, ?it/s]

Accuracy before training is 0.092 
-----------


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 1 is 11.059312105178833 sec
-----------
Accuracy for epoch 1 is 0.853 


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Time for epoch 8 is 10.830015897750854 sec
-----------
Accuracy for epoch 8 is 0.892 


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 9 is 10.786216974258423 sec
-----------
Accuracy for epoch 9 is 0.891 


100%|██████████| 1875/1875 [00:10<00:00, 176.40it/s]


Time for epoch 10 is 10.726639032363892 sec
-----------
Accuracy for epoch 10 is 0.9 


In [691]:
model_strong_defense.train(data_set, X_test[:1000],y_test[:1000], 10)

  0%|          | 0/1875 [00:00<?, ?it/s]

Accuracy before training is 0.082 
-----------


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 1 is 11.288469076156616 sec
-----------
Accuracy for epoch 1 is 0.85 


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Time for epoch 8 is 10.911808729171753 sec
-----------
Accuracy for epoch 8 is 0.889 


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 9 is 10.867736577987671 sec
-----------
Accuracy for epoch 9 is 0.891 


100%|██████████| 1875/1875 [00:10<00:00, 180.67it/s]


Time for epoch 10 is 10.475807905197144 sec
-----------
Accuracy for epoch 10 is 0.901 


In [703]:
model_average_defense.train(data_set, X_test[:1000],y_test[:1000], 10)

  0%|          | 0/1875 [00:00<?, ?it/s]

Accuracy before training is 0.124 
-----------


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 1 is 11.774292945861816 sec
-----------
Accuracy for epoch 1 is 0.863 


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Time for epoch 8 is 10.700856447219849 sec
-----------
Accuracy for epoch 8 is 0.894 


  0%|          | 0/1875 [00:00<?, ?it/s]

Time for epoch 9 is 10.916675090789795 sec
-----------
Accuracy for epoch 9 is 0.898 


100%|██████████| 1875/1875 [00:10<00:00, 172.67it/s]


Time for epoch 10 is 10.954395055770874 sec
-----------
Accuracy for epoch 10 is 0.885 


In [692]:
def DOC (x_adv_list,x_list):
    N=len(x_adv_list)
    sum=0
    for i in range (N):        
        sum+=100*np.linalg.norm(np.reshape(x_adv_list[i] - x_list[i], -1),ord=1)/np.linalg.norm(np.reshape(x_list[i], -1),ord=1)
    return(sum/N)

def succes_rate (success_list):
    success=len([i for i in success_list if i==True])
    total=len(success_list)
    return(success/total)

Test de la resistance du modèle 

In [693]:
def attack_performances_computer(model_to_attack,predicting_model,attack, image_list, labels,epsilon):
    model_to_attack=TensorFlowModel(model_to_attack , bounds=(0, 255))
   
    success_on_attacked_model=[]
    success_on_predicting_model=[]
    adv_list=[]
    for i,image in enumerate(tqdm(image_list,position=0)):
        image = np.asarray(image)[:
                                  ,:,:3].astype('float32')
        image = convert_to_tensor(np.expand_dims(image,axis=0))
        label=labels[i]
        label = tf.convert_to_tensor(np.array([label]))
        _, clipped, is_adv = attack(model_to_attack,image,label,epsilons=epsilon)
        success_on_attacked_model.append(bool(is_adv[0][0]))
        adv_list.append(np.array(clipped[0][0]))
        prediction=predicting_model.predict(np.expand_dims(clipped[0][0],axis=0))
        if np.argmax(prediction)!=labels[i]:
            success_on_predicting_model.append(True)
        else:
            success_on_predicting_model.append(False)
            
        
    DOC_attack=DOC(adv_list,image_list)
    SR_on_attacked_model=succes_rate(success_on_attacked_model)
    SR_on_predicting_model=succes_rate(success_on_predicting_model)
    return(DOC_attack,SR_on_attacked_model,SR_on_predicting_model)


In [694]:
def attack_runner(model_to_attack,predicting_model,image_list, labels_list, epislons_list):   
    labels_list=list(map(np.argmax,labels_list))
    
    attack_FGSM=LinfFastGradientAttack()
    
   

    SR_FGSM_dic={}  
    SR_FGSM_dic_with_defense={}


    for epsilon in epislons_list:
        print('======'+'epislon: '+str(epsilon)+'======')
        DOC_FGSM,SR_FGSM,SR_FGSM_with_defense=attack_performances_computer(model_to_attack,predicting_model,attack_FGSM, image_list, labels_list,[epsilon])
              
        SR_FGSM_dic[DOC_FGSM]=SR_FGSM
        SR_FGSM_dic_with_defense[DOC_FGSM]=SR_FGSM_with_defense
        
    return(SR_FGSM_dic,SR_FGSM_dic_with_defense)

In [695]:
SR_FGSM_dic_with_defense,_=attack_runner(model_with_def,model_with_def,X_test[7000:7500], y_test[7000:7500], [0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1])

  0%|          | 0/500 [00:00<?, ?it/s]

======epislon: 0.0005======


  1%|          | 3/500 [00:00<00:18, 26.28it/s]

======epislon: 0.001======


  1%|          | 3/500 [00:00<00:17, 28.98it/s]

...

  1%|          | 4/500 [00:00<00:16, 29.46it/s]

======epislon: 0.5======


  1%|          | 3/500 [00:00<00:18, 27.31it/s]

======epislon: 1======


100%|██████████| 500/500 [00:18<00:00, 26.32it/s]


In [696]:
SR_FGSM_dic_without_defense,_=attack_runner(model_without_def,model_without_def,X_test[7000:7500], y_test[7000:7500], [0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1])

  0%|          | 2/500 [00:00<00:27, 18.21it/s]

======epislon: 0.0005======


  1%|          | 3/500 [00:00<00:17, 27.65it/s]

======epislon: 0.001======


  1%|          | 3/500 [00:00<00:19, 25.27it/s]

...

  1%|          | 3/500 [00:00<00:18, 27.60it/s]

======epislon: 0.5======


  1%|          | 3/500 [00:00<00:17, 27.93it/s]

======epislon: 1======


100%|██████████| 500/500 [00:19<00:00, 26.21it/s]


In [697]:
SR_FGSM_dic_slight_defense,_=attack_runner(model_slight_defense,model_slight_defense,X_test[7000:7500], y_test[7000:7500], [0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1])

  0%|          | 2/500 [00:00<00:27, 18.13it/s]

======epislon: 0.0005======


  1%|          | 3/500 [00:00<00:19, 25.17it/s]

======epislon: 0.001======


  1%|          | 3/500 [00:00<00:19, 25.84it/s]

...

  1%|          | 3/500 [00:00<00:17, 27.64it/s]

======epislon: 0.5======


  1%|          | 3/500 [00:00<00:17, 27.76it/s]

======epislon: 1======


100%|██████████| 500/500 [00:18<00:00, 26.73it/s]


In [698]:
SR_FGSM_dic_strong_defense,_=attack_runner(model_strong_defense,model_strong_defense,X_test[7000:7500], y_test[7000:7500], [0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1])

  0%|          | 2/500 [00:00<00:27, 18.10it/s]

======epislon: 0.0005======


  1%|          | 3/500 [00:00<00:18, 27.60it/s]

======epislon: 0.001======


  1%|          | 3/500 [00:00<00:20, 23.86it/s]

...

  1%|          | 3/500 [00:00<00:19, 25.17it/s]

======epislon: 0.5======


  1%|          | 3/500 [00:00<00:17, 27.71it/s]

======epislon: 1======


100%|██████████| 500/500 [00:18<00:00, 26.44it/s]


In [ ]:
SR_FGSM_dic_average_defense,_=attack_runner(model_average_defense,model_average_defense,X_test[7000:7500], y_test[7000:7500], [0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1])

  0%|          | 2/500 [00:00<00:31, 15.90it/s]

======epislon: 0.0005======


  1%|          | 3/500 [00:00<00:18, 27.58it/s]

======epislon: 0.001======


  1%|          | 3/500 [00:00<00:23, 20.84it/s]

======epislon: 0.005======


 50%|█████     | 251/500 [00:10<00:09, 26.20it/s]

In [ ]:
plt.plot(list(SR_FGSM_dic_with_defense.keys()),list(SR_FGSM_dic_with_defense.values()),label='c=0.5')
plt.plot(list(SR_FGSM_dic_without_defense.keys()),list(SR_FGSM_dic_without_defense.values()),label='c=0')
plt.plot(list(SR_FGSM_dic_slight_defense.keys()),list(SR_FGSM_dic_slight_defense.values()),label='c=0.25')
plt.plot(list(SR_FGSM_dic_strong_defense.keys()),list(SR_FGSM_dic_strong_defense.values()),label='c=0.75')
plt.plot(list(SR_FGSM_dic_average_defense.keys()),list(SR_FGSM_dic_average_defense.values()),label='c=0.6')
plt.xscale('log')
plt.grid(True,which="both", linestyle='--')
plt.title('Successs rate for different degrees of change \n on Mnist images for the effnet model',fontsize=11)
plt.xlabel('DOC (%)')
plt.ylabel('SR')
plt.legend(loc='upper left')